In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [16]:
!hdfs dfs -rm -r -f /tmp/output

In [17]:
!hdfs dfs -mkdir /tmp/output

In [18]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/output/

In [19]:
%%hive

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.089 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.067 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 0.645 seconds


In [25]:
%%hive

DROP TABLE IF EXISTS resultado;

CREATE TABLE resultado
AS 
SELECT mayus
FROM(
    SELECT c1,
            collect_list(UPPER(letras)) AS mayus
    FROM tbl0
    LATERAL VIEW explode(c5) tbl0 AS letras
    GROUP BY c1
) t0
;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM resultado;

DROP TABLE IF EXISTS resultado;
OK
Time taken: 0.198 seconds
CREATE TABLE resultado
AS 
SELECT mayus
FROM(
    SELECT c1,
            collect_list(UPPER(letras)) AS mayus
    FROM tbl0
    LATERAL VIEW explode(c5) tbl0 AS letras
    GROUP BY c1
) t0
;
Query ID = root_20200118230634_72c02737-153c-4b26-be7d-9620bf3df920
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579385130637_0006, Tracking URL = http://282e0d2d9a5d:8088/proxy/application_1579385130637_0006/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579385130637_0006
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01

In [26]:
!hdfs dfs -copyToLocal /tmp/output output;

In [27]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.189 seconds


In [28]:
%hive_quit